In [ ]:
from bs4 import BeautifulSoup
from pandas import ExcelWriter
import pandas as pd
from tqdm import tqdm
import json
import ast
from tqdm import tqdm

In [ ]:
def xml2Excel(inDirectory = None, outFileName = None, outDirectory = None):
    path = inDirectory
    rawText = open(inDirectory).read()
    soup = BeautifulSoup(rawText, 'xml')
    rawTexts = []
    tags = []
    for record in soup.find_all('RECORD'):
        recordId = record["ID"]
        text = record.text
        rawTexts.append(text)
        start = 0
        tag = []
        for phi in record.find_all('PHI'):
            start = text.find(phi.text, start)
            end = start + len(phi.text)
            tagType = phi['TYPE']
            tag.append({'start': str(start), 'end': str(end), 'text': phi.text, 'TYPE': tagType})
        tags.append(tag)
    print(len(rawTexts), len(tags))
    fullPath = outDirectory + outFileName
    writer = ExcelWriter(outFileName)
    df = pd.DataFrame({'text': rawTexts, 'tags' : tags})
    df.to_excel(writer,'Sheet1', index = False)
    writer.close()

In [ ]:
xml2Excel(inDirectory='/content/drive/MyDrive/i2b2_2006/deid_surrogate_test_all_version2.xml', outDirectory='.',
         outFileName='i2b2-2006Test.xlsx')

220 220


In [ ]:
xml2Excel(inDirectory='/content/drive/MyDrive/i2b2_2006/deid_surrogate_train_all_version2.xml', outDirectory='.',
         outFileName='i2b2-2006Train.xlsx')

669 669


In [ ]:
xml2Excel(inDirectory='/content/drive/MyDrive/i2b2_2006/deid_surrogate_test_all_groundtruth_version2.xml', outDirectory='.',
         outFileName='i2b2-2014_validation.xlsx')

220 220


<ipython-input-2-b56a9d18e383>:24: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [ ]:
dfTrain = pd.read_excel('/content/i2b2-2006Train.xlsx')
dfTest = pd.read_excel('/content/i2b2-2006Test.xlsx')
dfValidation = pd.read_excel('/content/i2b2-2014_validation.xlsx')

In [ ]:
type(dfTrain['text'][0])

str

In [ ]:
type(dfTest['text'][0])

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
dfTrain

text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [ ]:
def func(txt, tgt):
  words = []
  ctag  = []
  j = 0
  for i in tgt:
    k = int(i['start'])
    l = int(i['end'])
    for tt in txt[j:k].strip().split(" "):
      tt = tt.replace("\n", '')
      tt = tt.replace("\n", '').strip()
      if tt != '':
        words.append(tt)
        ctag.append("O")
    info = i["text"].split(" ")
    ctag.append(f"B-{i['TYPE']}")
    for _ in info[1:]:
      ctag.append(f"I-{i['TYPE']}")
    words += [ii.strip() for ii in info]
    j = l
  for tt in txt[j:len(txt)].strip().split(" "):
    tt = tt.replace("\n", '')
    tt = tt.replace("\n", '').strip()
    if tt != '':
      words.append(tt)
      ctag.append("O")
  return words, ctag

In [ ]:
# Find all instances where the content does not end with ']'
rows_to_remove = dfTrain[~dfTrain['tags'].str.endswith(']')].index

# Remove those rows from the DataFrame
dfTrain = dfTrain.drop(index=rows_to_remove)

# Reset the index if needed
dfTrain = dfTrain.reset_index(drop=True)



In [ ]:
df_train = {'Doc #' : [], 'Words' : [], 'Tags' : [],'segment #' : []}

for i in tqdm(range(dfTrain.shape[0])):
  txt = dfTrain .iloc[i]['text']
  tgt = ast.literal_eval(dfTrain .iloc[i]['tags'])
  a,b = func(txt, tgt)
  df_train["Words"] += a
  df_train["Tags"] += b
  df_train['Doc #'] += [f"Doc-{i}"]* len(a)
  x = int(len(a)/100)
  y = len(a)%100
  for j in range(x):
    df_train['segment #'] += [f"segment {i} {j}"]*100
  df_train['segment #'] += [f"segment {i} {j+1}"]*y
df_train = pd.DataFrame(df_train)

100%|██████████| 114/114 [00:00<00:00, 967.93it/s] 


In [ ]:
df_train

,Doc #,Words,Tags,segment #
0,Doc-0,672921663,B-ID,segment 0 0
1,Doc-0,ELMVH,B-HOSPITAL,segment 0 0
2,Doc-0,09884732,B-ID,segment 0 0
3,Doc-0,882560,B-ID,segment 0 0
4,Doc-0,5/27,B-DATE,segment 0 0
...,...,...,...,...
55760,Doc-113,",",I-LOCATION,segment 113 3
55761,Doc-113,Chandigarh,I-LOCATION,segment 113 3
55762,Doc-113,.\n[,O,segment 113 3
55763,Doc-113,report_end,O,segment 113 3


In [ ]:
df_test = {'Doc #' : [], 'Words' : [], 'Tags' : [],'segment #' : []}

for i in tqdm(range(dfTest.shape[0])):
  txt = dfTest .iloc[i]['text']
  tgt = ast.literal_eval(dfTest .iloc[i]['tags'])
  a,b = func(txt, tgt)
  df_test["Words"] += a
  df_test["Tags"] += b
  df_test['Doc #'] += [f"Doc-{i}"]* len(a)
  x = int(len(a)/100)
  y = len(a)%100
  for j in range(x):
    df_test['segment #'] += [f"segment {i} {j}"]*100
  df_test['segment #'] += [f"segment {i} {j+1}"]*y
df_test = pd.DataFrame(df_test)

100%|██████████| 220/220 [00:00<00:00, 667.25it/s]


In [ ]:
df_train

,Doc #,Words,Tags,segment #
0,Doc-0,672921663,B-ID,segment 0 0
1,Doc-0,ELMVH,B-HOSPITAL,segment 0 0
2,Doc-0,09884732,B-ID,segment 0 0
3,Doc-0,882560,B-ID,segment 0 0
4,Doc-0,5/27,B-DATE,segment 0 0
...,...,...,...,...
55760,Doc-113,,I-LOCATION,segment 113 3
55761,Doc-113,Chandigarh,I-LOCATION,segment 113 3
55762,Doc-113,\n,O,segment 113 3
55763,Doc-113,report_end,O,segment 113 3


In [ ]:
def remove(x):
    x['Words'] = x['Words'].str.replace(r'^[\/;,.:\[\]*]+|[\/;,.:\[\]*]+$', '')
    x['Words'] = x['Words'].replace('\\n\\n', '')
    x['Words'] = x['Words'].replace('\\n\\n\\n', '')
    x['Words'] = x['Words'].replace('\\n\\n\\n\\n', '')
    x['Words'] = x['Words'].replace('\\n', '')
    x['Words'] = x['Words'].str.replace(r'^[\/;,.:\[\]*]+|[\/;,.:\[\]*]+$', '')
    x= x.dropna()
    return x


In [ ]:
df_train = df_train.apply(remove)
df_test = df_test.apply(remove)

In [ ]:
df_train.to_csv('Train_n2c2_2006.csv')
df_test.to_csv('Test_n2c2_2006.csv')
df_test.to_csv('val_n2c2_2006.csv')